# Medallion Architecture - Demo

**Cel szkoleniowy:** Zrozumienie koncepcji architektury medalionowej (Bronze/Silver/Gold) i zasad projektowania data lakehouse.

**Zakres tematyczny:**
- Bronze / Silver / Gold - logika warstw
- ETL vs ELT approach
- Zasady projektowania pipeline'ów
- Partitioning strategy
- Audyt i lineage - metadane w każdym kroku
- Data quality w kontekście warstw

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Lakehouse & Delta Lake
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie architektury medalionowej jako design pattern dla data lakehouse.

**Podstawowe pojęcia:**
- **Medallion Architecture**: Wielowarstwowa architektura danych (Bronze → Silver → Gold)
- **Bronze Layer**: Raw data landing zone - dane bez transformacji, tylko audit metadata
- **Silver Layer**: Cleansed and conformed data - deduplikacja, walidacja, standardizacja
- **Gold Layer**: Business-level aggregates - modele KPI, reporty, ML features

**Dlaczego to ważne?**
Medallion architecture zapewnia separation of concerns, jasne SLA per warstwa, incremental processing, oraz data quality gates. Umożliwia różne tempo procesowania (Bronze: real-time, Silver: hourly, Gold: daily) i różne retention policies per warstwa.

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog jako domyślny
spark.sql(f"USE CATALOG {CATALOG}")

# Ścieżki do danych źródłowych
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"

print(f"\n=== Ścieżki do danych ===")
print(f"Orders: {ORDERS_JSON}")
print(f"Customers: {CUSTOMERS_CSV}")

---

## Sekcja 1: Koncepcja Medallion Architecture

**Wprowadzenie teoretyczne:**

Medallion Architecture dzieli data lakehouse na trzy warstwy o rosnącej jakości danych. Każda warstwa ma określone SLA, retention policy i data quality requirements.

**Kluczowe pojęcia:**
- **Bronze (Raw)**: Append-only, immutable landing zone. Dane "as-is" z systemu źródłowego + audit metadata (ingestion timestamp, source file, version)
- **Silver (Cleansed)**: Validated, deduplicated, standardized. Business rules enforcement, schema evolution, data quality checks
- **Gold (Curated)**: Aggregated, denormalized, business-level. KPI models, reporting tables, ML features, star schema

**Zastosowanie praktyczne:**
- Separacja odpowiedzialności: data engineers (Bronze/Silver), analytics engineers (Gold)
- Incremental processing: tylko nowe/zmienione dane propagowane przez warstwy
- Debug-friendly: możliwość reprocessingu Silver/Gold z Bronze bez re-ingestion

### Przykład 1.1: Bronze Layer - Raw Data Landing

**Cel:** Utworzenie Bronze layer z audit metadata

**Podejście:**
1. Wczytanie surowych danych z JSON
2. Dodanie audit columns: ingest_timestamp, source_file, ingested_by
3. Zapis do Bronze schema bez transformacji biznesowych

In [0]:
# Przykład 1.1 - Bronze Layer

spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Wczytaj surowe dane orders
orders_raw = (
    spark.read
    .format("json")
    .option("multiLine", "true")
    .load(ORDERS_JSON)
)

print("=== Surowe dane (schema) ===")
orders_raw.printSchema()

# Dodaj audit metadata (Bronze best practice)
orders_bronze = (
    orders_raw
    .withColumn("ingest_timestamp", F.current_timestamp())
    .withColumn("source_file", F.input_file_name())
    .withColumn("ingested_by", F.lit(raw_user))
    .withColumn("bronze_version", F.lit(1))
)

print("\n=== Bronze layer z audit metadata ===")
display(orders_bronze.limit(3))

# Zapisz do Bronze schema
bronze_table = f"{BRONZE_SCHEMA}.orders_bronze"

(
    orders_bronze
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(bronze_table)
)

print(f"\n✓ Utworzono Bronze table: {bronze_table}")
print(f"Liczba rekordów: {spark.table(bronze_table).count()}")

**Wyjaśnienie:**

Bronze layer:
- **Immutable**: Dane zapisane "as-is" bez modyfikacji wartości biznesowych
- **Audit trail**: Każdy rekord ma metadata: kiedy, skąd, przez kogo został załadowany
- **Append-only**: Idealnie nadaje się do incremental loads z COPY INTO lub Auto Loader
- **Retention**: Często długa (lata) jako źródło prawdy do reprocessingu

**Struktura danych źródłowych (orders_batch.json)**:
- `order_id`: String (np. "ORD00000001")
- `customer_id`: String (np. "CUST005909")
- `order_datetime`: Timestamp (np. "2024-12-31T23:56:00")
- `total_amount`: Double - całkowita wartość zamówienia
- `payment_method`: String - metoda płatności

### Przykład 1.2: Silver Layer - Cleansed & Validated

**Cel:** Transformacja Bronze → Silver z data quality checks

In [0]:
# Przykład 1.2 - Silver Layer

spark.sql(f"USE SCHEMA {SILVER_SCHEMA}")

# Wczytaj dane z Bronze
orders_bronze_df = spark.table(bronze_table)

# Silver transformations: cleaning, validation, standardization
orders_silver = (
    orders_bronze_df
    # Deduplikacja po kluczu biznesowym
    .dropDuplicates(["order_id"])
    
    # Walidacja: usuń rekordy z NULL w kluczowych kolumnach
    .filter(F.col("order_id").isNotNull())
    .filter(F.col("customer_id").isNotNull())
    
    # Walidacja biznesowa: total_amount > 0
    .filter(F.col("total_amount") > 0)
    
    # Standaryzacja dat - użyj order_datetime zamiast order_date
    .withColumn("order_date", F.to_date(F.col("order_datetime")))
    
    # Standaryzacja tekstów
    .withColumn("payment_method", F.upper(F.trim(F.col("payment_method"))))
    
    # Dodaj Silver metadata
    .withColumn("silver_processed_timestamp", F.current_timestamp())
    .withColumn("data_quality_flag", F.lit("VALID"))
)

print("=== Silver layer - cleansed data ===")
display(orders_silver.limit(5))

# Zapisz do Silver schema
silver_table = f"{SILVER_SCHEMA}.orders_silver"

(
    orders_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(silver_table)
)

bronze_count = orders_bronze_df.count()
silver_count = spark.table(silver_table).count()

print(f"\n✓ Utworzono Silver table: {silver_table}")
print(f"Bronze records: {bronze_count}")
print(f"Silver records: {silver_count}")
print(f"Filtered out: {bronze_count - silver_count} records")

**Wyjaśnienie:**

Silver layer:
- **Data Quality**: Walidacja biznesowa (total_amount > 0), walidacja schematów (NOT NULL)
- **Deduplikacja**: Usunięcie duplikatów po kluczu biznesowym
- **Standardizacja**: Ujednolicenie formatów (daty, teksty, case sensitivity)
- **Incremental friendly**: Można używać MERGE dla slowly changing dimensions

**Nota**: W tym przykładzie transformujemy `order_datetime` (timestamp) na `order_date` (date) dla łatwiejszego partycjonowania i agregacji w Gold layer.

### Przykład 1.3: Gold Layer - Business Aggregates

**Cel:** Utworzenie Gold layer z KPI dla analityki i raportowania

In [0]:
# Przykład 1.3 - Gold Layer: Daily Order Summary

spark.sql(f"USE SCHEMA {GOLD_SCHEMA}")

# Wczytaj dane z Silver
orders_silver_df = spark.table(silver_table)

# Gold aggregation: Daily order summary
daily_summary = (
    orders_silver_df
    .groupBy("order_date", "order_status")
    .agg(
        F.count("order_id").alias("total_orders"),
        F.sum("total_amount").alias("total_revenue"),
        F.avg("total_amount").alias("avg_order_value"),
        F.min("total_amount").alias("min_order_value"),
        F.max("total_amount").alias("max_order_value"),
        F.countDistinct("customer_id").alias("unique_customers")
    )
    .withColumn("gold_created_timestamp", F.current_timestamp())
    .orderBy("order_date", "order_status")
)

print("=== Gold layer - Daily Order Summary ===")
display(daily_summary)

# Zapisz do Gold schema
gold_table = f"{GOLD_SCHEMA}.daily_order_summary"

(
    daily_summary
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(gold_table)
)

print(f"\n✓ Utworzono Gold table: {gold_table}")
print(f"Liczba agregowanych dni: {spark.table(gold_table).count()}")

**Wyjaśnienie:**

Gold layer:
- **Business-level**: KPI i metryki zgodne z business definitions
- **Denormalized**: Często szeroka tabela z joinami już wykonanymi (performance dla BI)
- **Aggregated**: Dane pre-aggregowane (daily, weekly, monthly) dla szybkich dashboardów
- **BI-ready**: Bezpośrednie źródło dla Power BI, Tableau, Looker

**Nota**: W tym przykładzie agregujemy zamówienia po dacie i metodzie płatności (payment_method). W rzeczywistych scenariuszach możesz agregować po innych wymiarach biznesowych jak region, kategoria produktu, czy segment klienta.

---

## Sekcja 2: ETL vs ELT Approach

**Wprowadzenie teoretyczne:**

Medallion architecture wspiera ELT (Extract-Load-Transform) approach, w przeciwieństwie do tradycyjnego ETL. Dane są najpierw ładowane do Bronze (Load), a potem transformowane w Silver/Gold (Transform).

**Kluczowe różnice:**
- **ETL**: Transform before load - dane są czyszczone poza data warehouse
- **ELT**: Load then transform - surowe dane w Bronze, transformacje w lakehouse
- **Zalety ELT**: Możliwość reprocessingu, data lineage, audit trail, schema evolution

### Przykład 2.1: ELT Pipeline - Incremental Processing

**Cel:** Demonstracja incremental ELT: nowe dane w Bronze → automatyczna propagacja do Silver/Gold

In [0]:
# Przykład 2.1 - Incremental ELT

# Symulacja: nowe dane przychodzą do Bronze
new_orders_data = [
    ("ORD99990001", "CUST009901", "PROD000001", "STORE001", "2025-01-20T10:00:00", 2, 175.00, 0, 350.00, "Credit Card"),
    ("ORD99990002", "CUST009902", "PROD000002", "STORE002", "2025-01-20T11:00:00", 1, 120.50, 0, 120.50, "Debit Card"),
    ("ORD99990003", "CUST009903", "PROD000003", "STORE003", "2025-01-21T09:00:00", 1, 499.99, 0, 499.99, "PayPal")
]

new_orders_df = spark.createDataFrame(
    new_orders_data,
    ["order_id", "customer_id", "product_id", "store_id", "order_datetime", "quantity", "unit_price", "discount_percent", "total_amount", "payment_method"]
)

# Dodaj audit metadata (Bronze standard)
new_orders_bronze = (
    new_orders_df
    .withColumn("ingest_timestamp", F.current_timestamp())
    .withColumn("source_file", F.lit("incremental_batch_2"))
    .withColumn("ingested_by", F.lit(raw_user))
    .withColumn("bronze_version", F.lit(2))
)

# Append do Bronze (ELT: Load first)
(
    new_orders_bronze
    .write
    .format("delta")
    .mode("append")
    .saveAsTable(bronze_table)
)

print(f"✓ Dodano {new_orders_df.count()} nowych rekordów do Bronze")
print(f"Bronze total: {spark.table(bronze_table).count()} records")

# Incremental Silver processing: tylko nowe Bronze records (version 2)
new_bronze_records = (
    spark.table(bronze_table)
    .filter(F.col("bronze_version") == 2)
)

# Apply Silver transformations
new_silver_records = (
    new_bronze_records
    .dropDuplicates(["order_id"])
    .filter(F.col("order_id").isNotNull())
    .filter(F.col("total_amount") > 0)
    .withColumn("order_date", F.to_date(F.col("order_datetime")))
    .withColumn("payment_method", F.upper(F.trim(F.col("payment_method"))))
    .withColumn("silver_processed_timestamp", F.current_timestamp())
    .withColumn("data_quality_flag", F.lit("VALID"))
)

# Append do Silver
(
    new_silver_records
    .write
    .format("delta")
    .mode("append")
    .saveAsTable(silver_table)
)

print(f"✓ Propagowano {new_silver_records.count()} rekordów do Silver")
print(f"Silver total: {spark.table(silver_table).count()} records")

# Gold: re-aggregate (lub incremental z MERGE)
# Dla uproszczenia: pełna re-agregacja
updated_daily_summary = (
    spark.table(silver_table)
    .groupBy("order_date", "payment_method")
    .agg(
        F.count("order_id").alias("total_orders"),
        F.sum("total_amount").alias("total_revenue"),
        F.avg("total_amount").alias("avg_order_value"),
        F.min("total_amount").alias("min_order_value"),
        F.max("total_amount").alias("max_order_value"),
        F.countDistinct("customer_id").alias("unique_customers")
    )
    .withColumn("gold_created_timestamp", F.current_timestamp())
)

(
    updated_daily_summary
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(gold_table)
)

print(f"✓ Zaktualizowano Gold layer")
print("\n=== Updated Gold Summary ===")
display(spark.table(gold_table).orderBy("order_date", "payment_method"))

**Wyjaśnienie:**

Incremental ELT pattern:
1. **Bronze**: Append nowych danych z wersjonowaniem (bronze_version)
2. **Silver**: Proces tylko nowe Bronze records (watermark lub version)
3. **Gold**: Re-aggregate lub MERGE dla affected partitions

W produkcji: używamy Delta Live Tables lub Structured Streaming dla automatic incrementality.

---

## Sekcja 3: Partitioning Strategy

**Wprowadzenie teoretyczne:**

Partycjonowanie to kluczowa decyzja architektoniczna w Medallion. Złe partycjonowanie powoduje small files problem lub inefficient queries.

**Zasady partycjonowania:**
- **Bronze**: Rzadko partycjonujemy (append-only, bulk operations)
- **Silver**: Partycjonowanie po dacie lub region dla incremental MERGE
- **Gold**: Partycjonowanie wg wymiarów zapytań (date, region, product_category)
- **Reguła**: Partycjonuj tylko jeśli tabela > 1 TB i partition size > 1 GB

### Przykład 3.1: Partycjonowanie Silver layer po dacie

**Cel:** Demonstracja partitioned table dla efektywnych incremental updates

In [0]:
# Przykład 3.1 - Partitioned Silver table

# Utwórz Silver table z partycjonowaniem po order_date
silver_partitioned_table = f"{SILVER_SCHEMA}.orders_silver_partitioned"

(
    spark.table(silver_table)
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("order_date")
    .option("overwriteSchema", "true")
    .saveAsTable(silver_partitioned_table)
)

print(f"✓ Utworzono partycjonowaną tabelę: {silver_partitioned_table}")

# Sprawdź partycje
partitions = spark.sql(f"SHOW PARTITIONS {silver_partitioned_table}")
print("\n=== Partycje ===")
display(partitions)

# DESCRIBE DETAIL - sprawdź partitioning columns
detail = spark.sql(f"DESCRIBE DETAIL {silver_partitioned_table}")
print("\n=== Detail (partitionColumns) ===")
display(detail.select("name", "partitionColumns", "numFiles"))

**Wyjaśnienie:**

Partycjonowanie:
- **Partition pruning**: Spark czyta tylko partycje spełniające predicate (WHERE order_date = '2025-01-20')
- **Incremental MERGE**: UPDATE/DELETE tylko affected partitions
- **Trade-off**: Zbyt dużo partycji (< 1 GB) powoduje small files problem

Best practice: Partycjonuj po kolumnie używanej w 80% zapytań (często: date, region).

---

## Best Practices

**Projektowanie warstw:**
- **Bronze**: Immutable, append-only. Długa retention (lata). Audit metadata obowiązkowe.
- **Silver**: Idempotentne transformacje. Możliwość reprocessingu z Bronze. MERGE dla SCD.
- **Gold**: Denormalized, aggregated. Partition wg business dimensions. Krótka retention (miesięcy).

**Data Quality:**
- **Bronze → Silver**: Walidacja schematów, business rules, deduplikacja
- **Silver → Gold**: Sprawdzenie completeness (czy wszystkie Bronze records dotarły?)
- **Expectations**: Używaj Delta Live Tables expectations (warn/drop/fail)

**Performance:**
- **Partycjonowanie**: Tylko dla dużych tabel (>1TB), partition size > 1GB
- **ZORDER**: Silver/Gold - po kluczu biznesowym lub często filtrowanych kolumnach
- **Auto Optimize**: Włącz dla Silver/Gold (częste małe zapisy)

**Governance:**
- **Unity Catalog**: Bronze/Silver/Gold jako osobne schemas z różnymi permissions
- **Lineage**: Używaj Delta Lake lineage do śledzenia Bronze → Silver → Gold
- **Retention**: Bronze (3-7 lat), Silver (1-2 lata), Gold (6-12 miesięcy)

---

## Troubleshooting

**Problem 1: Small files w Bronze**
**Objawy:** Setki małych plików Parquet po każdym ingeście

**Rozwiązanie:**
```python
# Włącz Auto Optimize dla Bronze
spark.sql(f"""
    ALTER TABLE {bronze_table} 
    SET TBLPROPERTIES (
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true'
    )
""")
```

**Problem 2: Silver processing zbyt wolny**
**Rozwiązanie:** Użyj incremental processing z watermark zamiast full table scan:
```python
# Proces tylko rekordy nowsze niż ostatni Silver timestamp
max_silver_ts = spark.table(silver_table).agg(F.max("ingest_timestamp")).collect()[0][0]
new_bronze = spark.table(bronze_table).filter(F.col("ingest_timestamp") > max_silver_ts)
```

**Problem 3: Gold re-aggregation trwa zbyt długo**
**Rozwiązanie:** Użyj MERGE zamiast overwrite dla incremental Gold:
```python
# Tylko affected dates
affected_dates = new_silver.select("order_date").distinct()
# DELETE affected partitions, INSERT new aggregates
```

---

## Podsumowanie

**W tym notebooku nauczyliśmy się:**

✅ **Medallion Architecture:**
- Bronze: Raw data landing zone z audit metadata (immutable, append-only)
- Silver: Cleansed, validated, deduplicated data (business rules enforcement)
- Gold: Business-level aggregates i KPI (BI-ready, denormalized)

✅ **ETL vs ELT:**
- ELT approach: Load first (Bronze), then Transform (Silver/Gold)
- Możliwość reprocessingu bez re-ingestion
- Incremental processing dla każdej warstwy

✅ **Partitioning Strategy:**
- Partycjonuj tylko duże tabele (>1TB)
- Partition size > 1GB dla uniknięcia small files
- Silver/Gold: partycjonowanie po dacie lub business dimensions

**Kluczowe wnioski:**
1. Medallion Architecture zapewnia separation of concerns i data quality gates
2. Każda warstwa ma określone SLA, retention policy i access patterns
3. Bronze jako immutable source of truth umożliwia reprocessing
4. Incremental processing jest kluczowy dla performance w dużej skali

**Następne kroki:**
- **Kolejny notebook**: 03_batch_streaming_load.ipynb - COPY INTO, Auto Loader, Structured Streaming
- **Warsztat praktyczny**: 01_delta_medallion_workshop.ipynb
- **Delta Live Tables**: Automatyczna implementacja Medallion z deklaratywnym API

---

## Cleanup

Opcjonalnie: usuń utworzone tabele Demo po zakończeniu ćwiczeń:

In [0]:
# Opcjonalne czyszczenie zasobów testowych
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone dane

# spark.sql(f"DROP TABLE IF EXISTS {bronze_table}")
# spark.sql(f"DROP TABLE IF EXISTS {silver_table}")
# spark.sql(f"DROP TABLE IF EXISTS {silver_partitioned_table}")
# spark.sql(f"DROP TABLE IF EXISTS {gold_table}")

# spark.catalog.clearCache()
# print("Zasoby zostały wyczyszczone")